# Using Iguazio's Streams API

The platform’s Streaming API enables working with data in the platform as streams. For more information, see the [streams overview](https://www.iguazio.com/docs/latest-release/concepts/streams/).

## Initialize

In [1]:
import v3io.dataplane

Create a dataplane client

In [2]:
v3io_client = v3io.dataplane.Client()

> **Note**: that you can pass to the client the `endpoint` and `access_key` parameters explicitly. The following code is equivalent to the default values:

``` python
from os import getenv
v3io_client = v3io.dataplane.Client(endpoint='http://v3io-webapi:8081',
                                    access_key=getenv('V3IO_ACCESS_KEY'))
```

All data in the platform is stored in user-defined data containers. In this case we use the predefined "users" container. For more information refer to [Data containers, collections, and objects documentation](https://www.iguazio.com/docs/latest-release/concepts/containers-collections-objects)

In [3]:
CONTAINER = 'users'

Data path where to store the kv table

In [4]:
from os import getenv, path

V3IO_USERNAME = getenv('V3IO_USERNAME')
STREAM_PATH = path.join(V3IO_USERNAME, 'examples', 'v3io', 'stream')

## Create a Stream

Creates and configures a new stream. The new stream is available immediately upon its creation.

You must configure the stream's shard count (the number of shards in the stream). You can increase the shard count at any time, but you cannot reduce it.

You can also set the stream's retention period (default is 24 hours). After this period elapses, when new records are added to the stream, the earliest ingested records are deleted.

In [5]:
response = v3io_client.stream.create(container=CONTAINER,
                                     stream_path=STREAM_PATH,
                                     shard_count=8)
print(response.status_code)

204


## Describe Stream

Retrieves a stream’s configuration, including the shard count and retention period.

In [6]:
response = v3io_client.stream.describe(container=CONTAINER,
                                       stream_path=STREAM_PATH)
print(response.status_code)

200


In [7]:
print(f'Shards: {response.output.shard_count}\nRetention period: {response.output.retention_period_hours} hours')

Shards: 8
Retention period: 24 hours


## Update Stream

Updates a stream's configuration by increasing its shard count. The changes are applied immediately.

> **Note**: You can increase the shard count at any time, but you cannot reduce it. From the platform’s perspective, there is virtually no cost to creating even thousands of shards. However, if you increase a stream’s shard count after its creation, new records with a previously used partition key will be assigned either to the same shard that was previously used for this partition key or to a new shard. For more information see [stream sharding and partitioning](https://www.iguazio.com/docs/latest-release/concepts/streams/#stream-sharding-and-partitioning)

> **Spark-Streaming Note**: To use the Spark Streaming API to consume records from new shards after a shard-count increase, you must first restart the consumer application.

In [8]:
response = v3io_client.stream.update(container=CONTAINER,
                                     stream_path=STREAM_PATH,
                                     shard_count=10)
print(response.status_code)

204


Describe the stream again to see the updated shard count

In [9]:
response = v3io_client.stream.describe(container=CONTAINER,
                                       stream_path=STREAM_PATH)
print(response.output.shard_count)

10


## Put Records

Adds records to a stream.

We'll define a function that will convert our text to english words

In [10]:
import re
regex_non_lower_case = re.compile('[^a-z]')

def text_to_words(text):
    # split into words
    words = text.split()
    # convert to lower case
    words = [w.lower() for w in words]
    # Keep only leters
    words = [regex_non_lower_case.sub('', w) for w in words]
    
    return words

text1 = "WOLF, meeting with a Lamb astray from the fold, resolved not to lay violent hands on him, but to find some plea to justify to the Lamb the Wolf’s right to eat him. He thus addressed him: “Sirrah, last year you grossly insulted me.” “Indeed,” bleated the Lamb in a mournful tone of voice, “I was not then born.” Then said the Wolf, “You feed in my pasture.” “No, good sir,” replied the Lamb, “I have not yet tasted grass.” Again said the Wolf, “You drink of my well.” “No,” exclaimed the Lamb, “I never yet drank water, for as yet my mother’s milk is both food and drink to me.” Upon which the Wolf seized him and ate him up, saying, “Well! I won’t remain supperless, even though you refute every one of my imputations.” The tyrant will always find a pretext for his tyranny."
words1 = text_to_words(text1)
len(words1)

146

Convert the list of words to a record. A record is a list of dictionaries, where for each dictionary the `data` key contains the record data.
We display the first 5 records:

In [11]:
records = [{'data': word} for word in words1]
print(records[:5])

[{'data': 'wolf'}, {'data': 'meeting'}, {'data': 'with'}, {'data': 'a'}, {'data': 'lamb'}]


In [12]:
response = v3io_client.stream.put_records(container=CONTAINER,
                                          stream_path=STREAM_PATH,
                                          records=records)
print(response.status_code)

200


Write another set of records to the stream

In [13]:
text2 = "A BAT who fell upon the ground and was caught by a Weasel pleaded to be spared his life. The Weasel refused, saying that he was by nature the enemy of all birds. The Bat assured him that he was not a bird, but a mouse, and thus was set free. Shortly afterwards the Bat again fell to the ground and was caught by another Weasel, whom he likewise entreated not to eat him. The Weasel said that he had a special hostility to mice. The Bat assured him that he was not a mouse, but a bat, and thus a second time escaped. It is wise to turn circumstances to good account."
response = v3io_client.stream.put_records(container=CONTAINER,
                                          stream_path=STREAM_PATH,
                                          records=[{'data': word} for word in text_to_words(text2)])

## Get Records

Multiple consumer instances can consume data from the same stream. A consumer retrieves records from a specific shard. It is recommended that you distribute the data consumption among several consumer instances (“workers”), assigning each instance one or more shards.

For each shard, the consumer should determine the location within the shard from which to begin consuming records. This can be the earliest ingested record, the end of the shard, the first ingested record starting at a specific time, or a specific record identified by its sequence number (a unique record identifier that is assigned by the platform based on the record’s ingestion time). The consumer first uses a seek operation to obtain the desired consumption location, and then provides this location as the starting point for its record consumption. The consumption operation returns the location of the next record to consume within the shard, and this location should be used as the location for a subsequent consumption operation, allowing for sequential record consumption.

In [14]:
shard_id = 0
seek_response = v3io_client.stream.seek(container=CONTAINER,
                                   stream_path=STREAM_PATH,
                                   shard_id=shard_id,
                                   seek_type='EARLIEST')
location = seek_response.output.location

We will read from the stream, by default `get_records` limits the number of records returned to 1,000. For the sake of this demonstration we will limit the number of returned records to 10 by setting the `limit` parameter.

In [15]:
get_response = v3io_client.stream.get_records(container=CONTAINER,
                                          stream_path=STREAM_PATH,
                                          shard_id=0,
                                          location=location,
                                          limit=10)
words_shard0 = [record.data.decode('utf-8') for record in get_response.output.records]
print(words_shard0)

['wolf', 'not', 'some', 'to', 'you', 'mournful', 'said', 'sir', 'again', 'exclaimed']


In [16]:
location = get_response.output.next_location

get_response = v3io_client.stream.get_records(container=CONTAINER,
                                          stream_path=STREAM_PATH,
                                          shard_id=0,
                                          location=location,
                                          limit=10)
words_shard0 = [record.data.decode('utf-8') for record in get_response.output.records]
print(words_shard0)

['yet', 'me', 'up', 'refute', 'find', 'upon', 'to', 'he', 'bat', 'a']


## Assigning Shard IDs

In the previous section, something does not look right. The words are not reordered, but many words are missing. The reason is that, by default, the platform assigns records to shards using a Round Robin algorithm. You can optionally assign a record to specific stream shard by specifying a related shard ID by setting the `shard_id` value, or associate the record with a specific partition key to ensure that similar records are assigned to the same shard (by setting the `partition_key` value). For more information see [stream sharding and partitioning](https://www.iguazio.com/docs/latest-release/concepts/streams/#stream-sharding-and-partitioning).

Let's ignore the existing shards (shards 0 through 9) and increase the number of shards

In [17]:
response = v3io_client.stream.update(container=CONTAINER,
                                     stream_path=STREAM_PATH,
                                     shard_count=12)

Write the first text to shard 10

In [18]:
response = v3io_client.stream.put_records(container=CONTAINER,
                                          stream_path=STREAM_PATH,
                                          records=[{'data': word, 'shard_id': 10} for word in words1])

And write the second text to shard 11

In [19]:
response = v3io_client.stream.put_records(container=CONTAINER,
                                          stream_path=STREAM_PATH,
                                          records=[{'data': word, 'shard_id': 11} for word in text_to_words(text2)])

Now, read from shard 10

In [20]:
shard_id = 10
seek_response = v3io_client.stream.seek(container=CONTAINER,
                                   stream_path=STREAM_PATH,
                                   shard_id=shard_id,
                                   seek_type='EARLIEST')
location = seek_response.output.location

get_response = v3io_client.stream.get_records(container=CONTAINER,
                                          stream_path=STREAM_PATH,
                                          shard_id=10,
                                          location=location)
words_shard10 = [record.data.decode('utf-8') for record in get_response.output.records]
print(f'{words_shard10[:10]}...')
if(words_shard10 == words1[:len(words_shard10)]):
    print("All words appear as expected")
else:
    print("Error, something is wrong")

['wolf', 'meeting', 'with', 'a', 'lamb', 'astray', 'from', 'the', 'fold', 'resolved']...
All words appear as expected


## Put Multiple Records

To get the highest possible throughput, we can send many requests towards the data layer and wait for all the responses to arrive (rather than send each request and wait for the response). The SDK supports this through batching. Any API call can be made through the client's built in `batch` object. The API call receives the exact same arguments it would normally receive (except for `raise_for_status`), and does not block until the response arrives. To wait for all pending responses, call `wait()` on the `batch` object:

`put_records` accepts up to a maximum of 1,000 records. If the records limit is exceeded, the request fails. Therefore, we would need to call `put_records` multiple times. 

In [21]:
all_text = [
    "WOLF, meeting with a Lamb astray from the fold, resolved not to lay violent hands on him, but to find some plea to justify to the Lamb the Wolf’s right to eat him. He thus addressed him: “Sirrah, last year you grossly insulted me.” “Indeed,” bleated the Lamb in a mournful tone of voice, “I was not then born.” Then said the Wolf, “You feed in my pasture.” “No, good sir,” replied the Lamb, “I have not yet tasted grass.” Again said the Wolf, “You drink of my well.” “No,” exclaimed the Lamb, “I never yet drank water, for as yet my mother’s milk is both food and drink to me.” Upon which the Wolf seized him and ate him up, saying, “Well! I won’t remain supperless, even though you refute every one of my imputations.” The tyrant will always find a pretext for his tyranny.",
    "A BAT who fell upon the ground and was caught by a Weasel pleaded to be spared his life. The Weasel refused, saying that he was by nature the enemy of all birds. The Bat assured him that he was not a bird, but a mouse, and thus was set free. Shortly afterwards the Bat again fell to the ground and was caught by another Weasel, whom he likewise entreated not to eat him. The Weasel said that he had a special hostility to mice. The Bat assured him that he was not a mouse, but a bat, and thus a second time escaped. It is wise to turn circumstances to good account.",
    "AN ASS having heard some Grasshoppers chirping, was highly enchanted; and, desiring to possess the same charms of melody, demanded what sort of food they lived on to give them such beautiful voices. They replied, “The dew.” The Ass resolved that he would live only upon dew, and in a short time died of hunger.",
    "A LION was awakened from sleep by a Mouse running over his face. Rising up angrily, he caught him and was about to kill him, when the Mouse piteously entreated, saying: “If you would only spare my life, I would be sure to repay your kindness.” The Lion laughed and let him go. It happened shortly after this that the Lion was caught by some hunters, who bound him by strong ropes to the ground. The Mouse, recognizing his roar, came and gnawed the rope with his teeth, and set him free, exclaiming: “You ridiculed the idea of my ever being able to help you, not expecting to receive from me any repayment of your favor; now you know that it is possible for even a Mouse to confer benefits on a Lion.”",
    "A CHARCOAL-BURNER carried on his trade in his own house. One day he met a friend, a Fuller, and entreated him to come and live with him, saying that they should be far better neighbors and that their housekeeping expenses would be lessened. The Fuller replied, “The arrangement is impossible as far as I am concerned, for whatever I should whiten, you would immediately blacken again with your charcoal.” Like will draw like.",
    "A FATHER had a family of sons who were perpetually quarreling among themselves. When he failed to heal their disputes by his exhortations, he determined to give them a practical illustration of the evils of disunion; and for this purpose he one day told them to bring him a bundle of sticks. When they had done so, he placed the faggot into the hands of each of them in succession, and ordered them to break it in pieces. They tried with all their strength, and were not able to do it. He next opened the faggot, took the sticks separately, one by one, and again put them into his sons’ hands, upon which they broke them easily. He then addressed them in these words: “My sons, if you are of one mind, and unite to assist each other, you will be as this faggot, uninjured by all the attempts of your enemies; but if you are divided among yourselves, you will be broken as easily as these sticks.”",
    "A BOY was hunting for locusts. He had caught a goodly number, when he saw a Scorpion, and mistaking him for a locust, reached out his hand to take him. The Scorpion, showing his sting, said: “If you had but touched me, my friend, you would have lost me, and all your locusts too!”",
    "A COCK, scratching for food for himself and his hens, found a precious stone and exclaimed: “If your owner had found thee, and not I, he would have taken thee up, and have set thee in thy first estate; but I have found thee for no purpose. I would rather have one barleycorn than all the jewels in the world.”",
    "THE BEASTS of the field and forest had a Lion as their king. He was neither wrathful, cruel, nor tyrannical, but just and gentle as a king could be. During his reign he made a royal proclamation for a general assembly of all the birds and beasts, and drew up conditions for a universal league, in which the Wolf and the Lamb, the Panther and the Kid, the Tiger and the Stag, the Dog and the Hare, should live together in perfect peace and amity. The Hare said, “Oh, how I have longed to see this day, in which the weak shall take their place with impunity by the side of the strong.” And after the Hare said this, he ran for his life.",
    "A WOLF who had a bone stuck in his throat hired a Crane, for a large sum, to put her head into his mouth and draw out the bone. When the Crane had extracted the bone and demanded the promised payment, the Wolf, grinning and grinding his teeth, exclaimed: “Why, you have surely already had a sufficient recompense, in having been permitted to draw out your head in safety from the mouth and jaws of a wolf.” In serving the wicked, expect no reward, and be thankful if you escape injury for your pains.",
    "A FISHERMAN skilled in music took his flute and his nets to the seashore. Standing on a projecting rock, he played several tunes in the hope that the fish, attracted by his melody, would of their own accord dance into his net, which he had placed below. At last, having long waited in vain, he laid aside his flute, and casting his net into the sea, made an excellent haul of fish. When he saw them leaping about in the net upon the rock he said: “O you most perverse creatures, when I piped you would not dance, but now that I have ceased you do so merrily.”",
    "A CARTER was driving a wagon along a country lane, when the wheels sank down deep into a rut. The rustic driver, stupefied and aghast, stood looking at the wagon, and did nothing but utter loud cries to Hercules to come and help him. Hercules, it is said, appeared and thus addressed him: “Put your shoulders to the wheels, my man. Goad on your bullocks, and never more pray to me for help, until you have done your best to help yourself, or depend upon it you will henceforth pray in vain.” Self-help is the best help. ",
    "THE ANTS were spending a fine winter’s day drying grain collected in the summertime. A Grasshopper, perishing with famine, passed by and earnestly begged for a little food. The Ants inquired of him, “Why did you not treasure up food during the summer?” He replied, “I had not leisure enough. I passed the days in singing.” They then said in derision: “If you were foolish enough to sing all the summer, you must dance supperless to bed in the winter.”",
    "A TRAVELER about to set out on a journey saw his Dog stand at the door stretching himself. He asked him sharply: “Why do you stand there gaping? Everything is ready but you, so come with me instantly.” The Dog, wagging his tail, replied: “O, master! I am quite ready; it is you for whom I am waiting.” The loiterer often blames delay on his more active friend. ",
    "A DOG, crossing a bridge over a stream with a piece of flesh in his mouth, saw his own shadow in the water and took it for that of another Dog, with a piece of meat double his own in size. He immediately let go of his own, and fiercely attacked the other Dog to get his larger piece from him. He thus lost both: that which he grasped at in the water, because it was a shadow; and his own, because the stream swept it away.",
    "A MOLE, a creature blind from birth, once said to his Mother: “I am sure than I can see, Mother!” In the desire to prove to him his mistake, his Mother placed before him a few grains of frankincense, and asked, “What is it?” The young Mole said, “It is a pebble.” His Mother exclaimed: “My son, I am afraid that you are not only blind, but that you have lost your sense of smell.”",
    "A HERDSMAN tending his flock in a forest lost a Bull-calf from the fold. After a long and fruitless search, he made a vow that, if he could only discover the thief who had stolen the Calf, he would offer a lamb in sacrifice to Hermes, Pan, and the Guardian Deities of the forest. Not long afterwards, as he ascended a small hillock, he saw at its foot a Lion feeding on the Calf. Terrified at the sight, he lifted his eyes and his hands to heaven, and said: “Just now I vowed to offer a lamb to the Guardian Deities of the forest if I could only find out who had robbed me; but now that I have discovered the thief, I would willingly add a full-grown Bull to the Calf I have lost, if I may only secure my own escape from him in safety.”",
    "A HARE one day ridiculed the short feet and slow pace of the Tortoise, who replied, laughing: “Though you be swift as the wind, I will beat you in a race.” The Hare, believing her assertion to be simply impossible, assented to the proposal; and they agreed that the Fox should choose the course and fix the goal. On the day appointed for the race the two started together. The Tortoise never for a moment stopped, but went on with a slow but steady pace straight to the end of the course. The Hare, lying down by the wayside, fell fast asleep. At last waking up, and moving as fast as he could, he saw the Tortoise had reached the goal, and was comfortably dozing after her fatigue. Slow but steady wins the race.",
    "THE POMEGRANATE and Apple-Tree disputed as to which was the most beautiful. When their strife was at its height, a Bramble from the neighboring hedge lifted up its voice, and said in a boastful tone: “Pray, my dear friends, in my presence at least cease from such vain disputings.”",
    "A FARMER placed nets on his newly-sown plowlands and caught a number of Cranes, which came to pick up his seed. With them he trapped a Stork that had fractured his leg in the net and was earnestly beseeching the Farmer to spare his life. “Pray save me, Master,” he said, “and let me go free this once. My broken limb should excite your pity. Besides, I am no Crane, I am a Stork, a bird of excellent character; and see how I love and slave for my father and mother. Look too, at my feathers—they are not the least like those of a Crane.” The Farmer laughed aloud and said, “It may be all as you say, I only know this: I have taken you with these robbers, the Cranes, and you must die in their company.” Birds of a feather flock together.",
    "ONE WINTER a Farmer found a Snake stiff and frozen with cold. He had compassion on it, and taking it up, placed it in his bosom. The Snake was quickly revived by the warmth, and resuming its natural instincts, bit its benefactor, inflicting on him a mortal wound. “Oh,” cried the Farmer with his last breath, “I am rightly served for pitying a scoundrel.” The greatest kindness will not bind the ungrateful.",
    "A YOUNG FAWN once said to his Mother, “You are larger than a dog, and swifter, and more used to running, and you have your horns as a defense; why, then, O Mother! do the hounds frighten you so?” She smiled, and said: “I know full well, my son, that all you say is true. I have the advantages you mention, but when I hear even the bark of a single dog I feel ready to faint, and fly away as fast as I can.” No arguments will give courage to the coward.",
    "A BEAR boasted very much of his philanthropy, saying that of all animals he was the most tender in his regard for man, for he had such respect for him that he would not even touch his dead body. A Fox hearing these words said with a smile to the Bear, “Oh! that you would eat the dead and not the living.”"
]

In [22]:
response = v3io_client.stream.describe(container=CONTAINER,
                                       stream_path=STREAM_PATH)
shard_count = response.output.shard_count

In [23]:
import functools
from operator import iconcat
records = []
for index, text in enumerate(all_text):
    records.append([{'data': word, 'partition_key': f'text_{index}'} for word in text_to_words(text)])

# Flatten the records list
records = functools.reduce(iconcat, records, [])
print(f'Total {len(records)} records to stream')

Total 2236 records to stream


In [24]:
max_records = 1000
for i in range(0, len(records), max_records):
    v3io_client.batch.stream.put_records(container=CONTAINER,
                                         stream_path=STREAM_PATH,
                                         records=records[i:i+max_records])

# wait for all writes to complete
responses = v3io_client.batch.wait()

AttributeError: 'function' object has no attribute 'put_records'

The looped `put_records` interface above will send all `put records` requests to the data layer in parallel. When `wait` is called, it will block until either all responses arrive (in which case it will return a `Responses` object, containing the `responses` of each call) or an error occurs - in which case an exception is thrown. You can pass `raise_for_status` to `wait`, and it behaves as explained above.

> Note: The `batch` object is stateful, so you can only create one batch at a time. However, you can create multiple parallel batches yourself through the client's `create_batch()` interface

## Delete Stream

Deletes a stream object along with all of its shards.

In [25]:
response = v3io_client.stream.delete(container=CONTAINER, stream_path=STREAM_PATH)
print(response.status_code)

204


Alternatively you can use the following commands:
``` python
import shutil
V3IO_STREAM_PATH = path.join(sep, 'v3io', CONTAINER, STREAM_PATH)
shutil.rmtree(V3IO_STREAM_PATH)
```

or

```
!rm -r $V3IO_STREAM_PATH
```